>Source: U.S. Energy Information Administration (Jan 2021) [Data Set](https://www.kaggle.com/mruanova/us-gasoline-and-diesel-retail-prices-19952021/settings)

- A1 = Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)
- A2 = Weekly U.S. All Grades Conventional Retail Gasoline Prices  (Dollars per Gallon)
- A3 = Weekly U.S. All Grades Reformulated Retail Gasoline Prices  (Dollars per Gallon)
- R1 = Weekly U.S. Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)
- R2 = Weekly U.S. Regular Conventional Retail Gasoline Prices  (Dollars per Gallon)
- R3 = Weekly U.S. Regular Reformulated Retail Gasoline Prices  (Dollars per Gallon)
- M1 = Weekly U.S. Midgrade All Formulations Retail Gasoline Prices  (Dollars per Gallon)
- M2 = Weekly U.S. Midgrade Conventional Retail Gasoline Prices  (Dollars per Gallon)
- M3 = Weekly U.S. Midgrade Reformulated Retail Gasoline Prices  (Dollars per Gallon)
- P1 = Weekly U.S. Premium All Formulations Retail Gasoline Prices  (Dollars per Gallon)
- P2 = Weekly U.S. Premium Conventional Retail Gasoline Prices  (Dollars per Gallon)
- P3 = Weekly U.S. Premium Reformulated Retail Gasoline Prices  (Dollars per Gallon)
- D1 = Weekly U.S. No 2 Diesel Retail Prices  (Dollars per Gallon)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
path = '../input/us-gasoline-and-diesel-retail-prices-19952021/PET_PRI_GND_DCUS_NUS_W.csv'
df = pd.read_csv(path, delimiter=',')
df.shape

In [ ]:
df.head(3)

In [ ]:
# Extracting month from date column.
df.Date = df.Date.apply(pd.to_datetime)
df['Day'] = df['Date'].apply(lambda x:x.day)
df['Month'] = df['Date'].apply(lambda x:x.month)
df['Year'] = df['Date'].apply(lambda x:x.year)
df.drop('Date',axis=1,inplace=True)
df.Month = df.Month.map({1:'JAN',2:'FEB',3:'MARCH',4:'APRIL',5:'MAY',6:'JUNE',7:'JULY',8:'AUG',9:'SEPT',10:'OCT',11:'NOV',12:'DEC'})
df.tail(3)

In [ ]:
# Creating dummy variables
dummies = pd.get_dummies(df[['Year','Day','Month']],drop_first=True)
df_dummies = pd.concat([df[['A1','A2','A3','R1','R2','R3','M1', 'M2','M3','P1','P2','P3']],dummies],axis=1)
target = df['D1']

# Splitting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_dummies,target,test_size=0.30)

# Standardizing the data
cols_to_std = ['A1','A2','A3','R1','R2','R3','M1', 'M2','M3','P1','P2','P3']
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train[cols_to_std])
X_train[cols_to_std] = scaler.transform(X_train[cols_to_std])
X_test[cols_to_std] = scaler.transform(X_test[cols_to_std])

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
model = XGBRegressor()
model.fit(X_train, y_train)
Y_pred = model.predict(X_test)
score = model.score(X_train, y_train)
print('Training Score:', score)
score = model.score(X_test, y_test)
print('Testing Score:', score)
output = pd.DataFrame({'Predicted':Y_pred})